[View in Colaboratory](https://colab.research.google.com/github/ryuwryyy/fastai_colab/blob/master/lesson6_rnn.ipynb)

In [7]:
# %reload_ext autoreload
# %autoreload 2
# %matplotlib inline
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
!pip install torchtext

!pip install bcolz
!pip install graphviz
!pip install sklearn_pandas
!pip install isoweek
!pip install pandas_summary
!pip install ipywidgets
!pip install fastai

from fastai.imports import *

gpg: keybox '/tmp/tmpjmo30223/pubring.gpg' created
gpg: /tmp/tmpjmo30223/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have s

plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.


In [0]:
from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

In [10]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

## Setup

We're going to download the collected works of Nietzsche to use as our data for this class.

In [12]:
%%bash
cd drive 
cd fastai
cd courses
cd dl1
cd data
pwd

/content/drive/fastai/courses/dl1/data


In [0]:
PATH='/content/drive/fastai/courses/dl1/data/'

In [18]:
# get_data("https://s3.amazonaws.com/text-datasets/nietzsche.txt", f'{PATH}nietzsche.txt')
text = open(f'{PATH}nietzsche.txt').read()
print('corpus length:', len(text))

corpus length: 611277


In [19]:
text[:400]

'{\\rtf1\\ansi\\ansicpg932\\cocoartf1504\\cocoasubrtf830\n{\\fonttbl\\f0\\fmodern\\fcharset0 Courier;}\n{\\colortbl;\\red255\\green255\\blue255;\\red0\\green0\\blue0;}\n{\\*\\expandedcolortbl;;\\cssrgb\\c0\\c0\\c0;}\n\\paperw11900\\paperh16840\\margl1440\\margr1440\\vieww10800\\viewh8400\\viewkind0\n\\deftab720\n\\pard\\pardeftab720\\sl280\\partightenfactor0\n\n\\f0\\fs24 \\cf2 \\expnd0\\expndtw0\\kerning0\n\\outl0\\strokewidth0 \\strokec2 PREFACE\\\n'

In [20]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 84


Sometimes it's useful to have a zero value in the dataset, e.g. for padding

In [21]:
chars.insert(0, "\0")

''.join(chars[1:-6])

'\n !"\'()*,-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]_abcdefghijklmnopqrstuv'

Map from chars to indices and back again

In [0]:
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

*idx* will be the data we use from now own - it simply converts all the characters to their index (based on the mapping above)

In [23]:
idx = [char_indices[c] for c in text]

idx[:10]

[82, 53, 73, 75, 61, 13, 53, 56, 69, 74]

In [24]:
''.join(indices_char[i] for i in idx[:70])

'{\\rtf1\\ansi\\ansicpg932\\cocoartf1504\\cocoasubrtf830\n{\\fonttbl\\f0\\fmoder'

## Three char model

### Create inputs

Create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters

In [0]:
cs=3
c1_dat = [idx[i]   for i in range(0, len(idx)-cs, cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-cs, cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-cs, cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-cs, cs)]

Our inputs

In [0]:
x1 = np.stack(c1_dat)
x2 = np.stack(c2_dat)
x3 = np.stack(c3_dat)

Our output

In [0]:
y = np.stack(c4_dat)

The first 4 inputs and outputs

In [28]:
x1[:4], x2[:4], x3[:4]

(array([82, 75, 53, 74]), array([53, 61, 56, 64]), array([73, 13, 69, 53]))

In [29]:
y[:4]

array([75, 53, 74, 56])

In [30]:
x1.shape, y.shape

((203758,), (203758,))

### Create and train model

Pick a size for our hidden state

In [0]:
n_hidden = 256

The number of latent factors to create (i.e. the size of the embedding matrix)

In [0]:
n_fac = 42

In [0]:
class Char3Model(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)

        # The 'green arrow' from our diagram - the layer operation from input to hidden
        self.l_in = nn.Linear(n_fac, n_hidden)

        # The 'orange arrow' from our diagram - the layer operation from hidden to hidden
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        
        # The 'blue arrow' from our diagram - the layer operation from hidden to output
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, c1, c2, c3):
        in1 = F.relu(self.l_in(self.e(c1)))
        in2 = F.relu(self.l_in(self.e(c2)))
        in3 = F.relu(self.l_in(self.e(c3)))
        
        h = V(torch.zeros(in1.size()).cuda())
        h = F.tanh(self.l_hidden(h+in1))
        h = F.tanh(self.l_hidden(h+in2))
        h = F.tanh(self.l_hidden(h+in3))
        
        return F.log_softmax(self.l_out(h))

In [0]:
md = ColumnarModelData.from_arrays('.', [-1], np.stack([x1,x2,x3], axis=1), y, bs=512)

In [0]:
m = Char3Model(vocab_size, n_fac).cuda()

In [0]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [0]:
opt = optim.Adam(m.parameters(), 1e-2)

In [38]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      2.028914   1.400064  



[array([1.40006])]

In [0]:
set_lrs(opt, 0.001)

In [48]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.783591   1.838515  



[array([1.83851])]

### Test model

In [0]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [41]:
get_next('y. ')

'T'

In [42]:
get_next('ppl')

'e'

In [43]:
get_next(' th')

'e'

In [44]:
get_next('and')

' '

## Our first RNN!

### Create inputs

This is the size of our unrolled RNN.

In [0]:
cs=8

For each of 0 through 7, create a list of every 8th character with that starting point. These will be the 8 inputs to out model.

In [0]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(len(idx)-cs)]

Then create a list of the next character in each of these series. This will be the labels for our model.

In [0]:
c_out_dat = [idx[j+cs] for j in range(len(idx)-cs)]

In [0]:
xs = np.stack(c_in_dat, axis=0)

In [49]:
xs.shape

(611269, 8)

In [0]:
y = np.stack(c_out_dat)

So each column below is one series of 8 characters from the text.

In [51]:
xs[:cs,:cs]

array([[82, 53, 73, 75, 61, 13, 53, 56],
       [53, 73, 75, 61, 13, 53, 56, 69],
       [73, 75, 61, 13, 53, 56, 69, 74],
       [75, 61, 13, 53, 56, 69, 74, 64],
       [61, 13, 53, 56, 69, 74, 64, 53],
       [13, 53, 56, 69, 74, 64, 53, 56],
       [53, 56, 69, 74, 64, 53, 56, 69],
       [56, 69, 74, 64, 53, 56, 69, 74]])

...and this is the next character after each sequence.

In [52]:
y[:cs]

array([69, 74, 64, 53, 56, 69, 74, 64])

### Create and train model

In [0]:
val_idx = get_cv_idxs(len(idx)-cs-1)

In [0]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)

In [0]:
class CharLoopModel(nn.Module):
    # This is an RNN!
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = F.relu(self.l_in(self.e(c)))
            h = F.tanh(self.l_hidden(h+inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [0]:
m = CharLoopModel(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [66]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.995524   2.054674  



[array([2.05467])]

In [0]:
set_lrs(opt, 0.001)

In [68]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.721706   1.719721  



[array([1.71972])]

In [0]:
class CharLoopConcatModel(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac+n_hidden, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = torch.cat((h, self.e(c)), 1)
            inp = F.relu(self.l_in(inp))
            h = F.tanh(self.l_hidden(inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [0]:
m = CharLoopConcatModel(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [0]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [72]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.85556    1.834462  



[array([1.83446])]

In [0]:
set_lrs(opt, 1e-4)

In [74]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.751862   1.748123  



[array([1.74812])]

### Test model

In [0]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [62]:
get_next('for thos')

'_'

In [63]:
get_next('part of ')

'Y'

In [64]:
get_next('queens a')

'3'

## RNN with pytorch

In [0]:
class CharRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        
        return F.log_softmax(self.l_out(outp[-1]), dim=-1)

In [0]:
m = CharRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [0]:
it = iter(md.trn_dl)
*xs,yt = next(it)

In [68]:
t = m.e(V(torch.stack(xs)))
t.size()

torch.Size([8, 512, 42])

In [69]:
ht = V(torch.zeros(1, 512,n_hidden))
outp, hn = m.rnn(t, ht)
outp.size(), hn.size()

(torch.Size([8, 512, 256]), torch.Size([1, 512, 256]))

In [70]:
t = m(*V(xs)); t.size()

torch.Size([512, 84])

In [85]:
fit(m, md, 4, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.819595   1.811873  
 21%|██▏       | 205/956 [00:06<00:24, 30.64it/s, loss=1.76]

    1      1.649484   1.650412  


    2      1.555515   1.579529  
    3      1.518035   1.535175  



[array([1.53517])]

In [0]:
set_lrs(opt, 1e-4)

In [87]:
fit(m, md, 2, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.451645   1.494163  
 21%|██        | 201/956 [00:07<00:26, 28.56it/s, loss=1.44]

    1      1.445542   1.489428  



[array([1.48943])]

### Test model

In [0]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [73]:
get_next('for thos')

'{'

In [0]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [75]:
get_next_n('for thos', 40)

'for thos{_E 1xYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY'

## Multi-output model

### Setup

Let's take non-overlapping sets of characters this time

In [0]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(0, len(idx)-cs-1, cs)]

Then create the exact same thing, offset by 1, as our labels

In [0]:
c_out_dat = [[idx[i+j] for i in range(cs)] for j in range(1, len(idx)-cs, cs)]

In [78]:
xs = np.stack(c_in_dat)
xs.shape

(76409, 8)

In [79]:
ys = np.stack(c_out_dat)
ys.shape

(76409, 8)

In [80]:
xs[:cs,:cs]

array([[82, 53, 73, 75, 61, 13, 53, 56],
       [69, 74, 64, 53, 56, 69, 74, 64],
       [58, 71, 62, 21, 15, 14, 53, 58],
       [70, 58, 70, 56, 73, 75, 61, 13],
       [17, 12, 16, 53, 58, 70, 58, 70],
       [56, 74, 76, 57, 73, 75, 61, 20],
       [15, 12,  1, 82, 53, 61, 70, 69],
       [75, 75, 57, 67, 53, 61, 12, 53]])

In [81]:
ys[:cs,:cs]

array([[53, 73, 75, 61, 13, 53, 56, 69],
       [74, 64, 53, 56, 69, 74, 64, 58],
       [71, 62, 21, 15, 14, 53, 58, 70],
       [58, 70, 56, 73, 75, 61, 13, 17],
       [12, 16, 53, 58, 70, 58, 70, 56],
       [74, 76, 57, 73, 75, 61, 20, 15],
       [12,  1, 82, 53, 61, 70, 69, 75],
       [75, 57, 67, 53, 61, 12, 53, 61]])

### Create and train model

In [0]:
val_idx = get_cv_idxs(len(xs)-cs-1)

In [0]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, ys, bs=512)

In [0]:
class CharSeqRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        return F.log_softmax(self.l_out(outp), dim=-1)

In [0]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [0]:
it = iter(md.trn_dl)
*xst,yt = next(it)

In [0]:
def nll_loss_seq(inp, targ):
    sl,bs,nh = inp.size()
    targ = targ.transpose(0,1).contiguous().view(-1)
    return F.nll_loss(inp.view(-1,nh), targ)

In [104]:
fit(m, md, 4, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      2.559035   2.361934  
    1      2.253459   2.162219  
    2      2.102857   2.049953  
    3      2.01354    1.979157  



[array([1.97916])]

In [0]:
set_lrs(opt, 1e-4)

In [106]:
fit(m, md, 1, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      1.965606   1.964351  



[array([1.96435])]

### Identity init!

In [0]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [89]:
m.rnn.weight_hh_l0.data.copy_(torch.eye(n_hidden))


    1     0     0  ...      0     0     0
    0     1     0  ...      0     0     0
    0     0     1  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      1     0     0
    0     0     0  ...      0     1     0
    0     0     0  ...      0     0     1
[torch.cuda.FloatTensor of size 256x256 (GPU 0)]

In [96]:
fit(m, md, 4, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      2.299149   2.127625  
    1      2.03591    1.979326  
    2      1.933386   1.902763  
    3      1.883065   1.874069  



[array([1.87407])]

In [0]:
set_lrs(opt, 1e-3)

In [98]:
fit(m, md, 4, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      1.790459   1.804721  
    1      1.776234   1.7974    
    2      1.766276   1.79309   
    3      1.758587   1.787752  



[array([1.78775])]

## Stateful model

### Setup

In [90]:
!pip install spacy 
from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

    100% |████████████████████████████████| 17.6MB 1.6MB/s 
    100% |████████████████████████████████| 92kB 14.4MB/s 
    100% |████████████████████████████████| 1.2MB 12.2MB/s 
    100% |████████████████████████████████| 51kB 18.8MB/s 
    100% |████████████████████████████████| 194kB 23.2MB/s 
    100% |████████████████████████████████| 71kB 22.4MB/s 
    100% |████████████████████████████████| 604kB 18.6MB/s 
    100% |████████████████████████████████| 389kB 22.8MB/s 
    100% |████████████████████████████████| 143kB 21.6MB/s 
  Running setup.py bdist_wheel for spacy ... - \ | / - \ | /

 - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/fb/00/28/75c85d5135e7d9a100639137d1847d41e914ed16c962d467e4
  Running setup.py bdist_wheel for murmurhash ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/b8/94/a4/f69f8664cdc1098603df44771b7fec5fd1b3d8364cdd83f512
  Running setup.py bdist_wheel for cymem ... - \ done
  Stored in directory: /content/.cache/pip/wheels/55/8d/4a/f6328252aa2aaec0b1cb906fd96a1566d77f0f67701071ad13
  Running setup.py bdist_wheel for preshed ... - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/8f/85/06/2d132fb649a6bbcab22487e4147880a55b0dd0f4b18fdfd6b5
  Running setup.py bdist_wheel for thinc ... - \ |

 - \ | / done
  Stored in directory: /content/.cache/pip/wheels/f8/b1/86/c92e4d36b690208fff8471711b85eaa6bc6d19860a86199a09
  Running setup.py bdist_wheel for msgpack-python ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/d5/de/86/7fa56fda12511be47ea0808f3502bc879df4e63ab168ec0406
Successfully built spacy murmurhash cymem preshed thinc pathlib ujson dill regex wrapt cytoolz msgpack-python
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.


In [101]:
PATH='/content/drive/fastai/courses/dl1/data/nietzsche/'
TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

nietzsche.txt  trn/  val/


In [102]:
%ls {PATH}trn

nietzsche.txt


In [103]:
TEXT = data.Field(lower=True, tokenize=list)
bs=64; bptt=8; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(1173, 57, 1, 601334)

### RNN

In [0]:
class CharSeqStatefulRnn(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        self.vocab_size = vocab_size
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [0]:
m = CharSeqStatefulRnn(md.nt, n_fac, 512).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [106]:
fit(m, md, 4, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.759435   1.761553  
 14%|█▍        | 165/1173 [00:02<00:12, 78.43it/s, loss=1.73]

    1      1.586691   1.590428  


    2      1.509047   1.514635  
    3      1.456913   1.46347   



[array([1.46347])]

In [108]:
set_lrs(opt, 1e-4)

fit(m, md, 4, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.389399   1.418022  
 14%|█▍        | 165/1173 [00:02<00:13, 75.12it/s, loss=1.42]

    1      1.385017   1.411296  


    2      1.382922   1.405069  
    3      1.385784   1.40059   



[array([1.40059])]

### RNN loop

In [0]:
# From the pytorch source

def RNNCell(input, hidden, w_ih, w_hh, b_ih, b_hh):
    return F.tanh(F.linear(input, w_ih, b_ih) + F.linear(hidden, w_hh, b_hh))

In [0]:
class CharSeqStatefulRnn2(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNNCell(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp = []
        o = self.h
        for c in cs: 
            o = self.rnn(self.e(c), o)
            outp.append(o)
        outp = self.l_out(torch.stack(outp))
        self.h = repackage_var(o)
        return F.log_softmax(outp, dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [0]:
m = CharSeqStatefulRnn2(md.nt, n_fac, 512).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [113]:
fit(m, md, 4, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.757587   1.750365  
 10%|█         | 119/1173 [00:02<00:23, 44.11it/s, loss=1.75]

    1      1.585627   1.594013  


    2      1.509411   1.516172  
    3      1.46629    1.469793  



[array([1.46979])]

### GRU

In [0]:
class CharSeqStatefulGRU(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.GRU(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [0]:
# From the pytorch source code - for reference

def GRUCell(input, hidden, w_ih, w_hh, b_ih, b_hh):
    gi = F.linear(input, w_ih, b_ih)
    gh = F.linear(hidden, w_hh, b_hh)
    i_r, i_i, i_n = gi.chunk(3, 1)
    h_r, h_i, h_n = gh.chunk(3, 1)

    resetgate = F.sigmoid(i_r + h_r)
    inputgate = F.sigmoid(i_i + h_i)
    newgate = F.tanh(i_n + resetgate * h_n)
    return newgate + inputgate * (hidden - newgate)

In [0]:
m = CharSeqStatefulGRU(md.nt, n_fac, 512).cuda()

opt = optim.Adam(m.parameters(), 1e-3)

In [117]:
fit(m, md, 6, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.634134   1.627754  
 14%|█▍        | 163/1173 [00:02<00:13, 75.99it/s, loss=1.6] 

    1      1.469983   1.466235  


    2      1.400803   1.394144  


    3      1.344681   1.348228  


    4      1.32058    1.317386  
    5      1.294916   1.298565  



[array([1.29856])]

In [0]:
set_lrs(opt, 1e-4)

In [119]:
fit(m, md, 3, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.221177   1.249778  
 14%|█▎        | 161/1173 [00:02<00:13, 74.96it/s, loss=1.26]

    1      1.219317   1.242545  
    2      1.215105   1.237456  



[array([1.23746])]

### Putting it all together: LSTM

In [0]:
from fastai import sgdr

n_hidden=512

In [0]:
class CharSeqStatefulLSTM(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [0]:
m = CharSeqStatefulLSTM(md.nt, n_fac, 512, 2).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)

In [0]:
os.makedirs(f'{PATH}models', exist_ok=True)

In [125]:
fit(m, md, 2, lo.opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.71842    1.65452   
 10%|█         | 118/1173 [00:02<00:22, 46.16it/s, loss=1.72]

    1      1.62126    1.554723  



[array([1.55472])]

In [126]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**4-1, lo.opt, F.nll_loss, callbacks=cb)

epoch      trn_loss   val_loss   
    0      1.45971    1.40309   
 11%|█         | 126/1173 [00:05<00:49, 21.35it/s, loss=1.54]

    1      1.505412   1.44295   


    2      1.393366   1.346028  


    3      1.533939   1.461758  


    4      1.46138    1.403365  


...

    10     1.419714   1.36739   


    11     1.383215   1.326291  


    12     1.343279   1.290007  


    13     1.301592   1.257772  
    14     1.274262   1.239197  



[array([1.2392])]

In [127]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**6-1, lo.opt, F.nll_loss, callbacks=cb)

epoch      trn_loss   val_loss   
    0      1.269279   1.23583   
 10%|█         | 121/1173 [00:05<00:52, 20.20it/s, loss=1.29]

    1      1.270894   1.232607  


    2      1.263501   1.230537  


    3      1.270197   1.227512  


    4      1.266815   1.222202  


...

    58     1.050266   0.979607  


    59     1.050603   0.978361  


    60     1.047582   0.976811  


    61     1.049601   0.975943  
    62     1.053668   0.975307  



[array([0.97531])]

### Test

In [0]:
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = m(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [130]:
get_next('for thos')

'e'

In [0]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [132]:
print(get_next_n('for thos', 400))

for those time\to what it science--words\preferents2 suphose\by the father, and\necessity, and disposed\and movement and double being life\flood. here, men, ansiques deal away of religion\as manifest sublime\and which they was to\sheen-man.--"no\hundred--date of democrity. we free more in\desire conceale.\\i1 the trouble ats\breaking nature sensible? and out others\occasions: what is\that is infredical, p
